In [2]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Query3-SQL').getOrCreate()
df = spark.read.load("hw5_problem2.parquet")
df.show(5)

+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
|                 _c0|                 _c1|                 _c2|     _c3|_c4|    _c5|    _c6|_c7|   _c8|
+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
|eefc2eb34a354ab7a...|2019-09-12T00:00:...|http://example.co...|user-041| CR|     IE|Android|201|0.2856|
|9ba55c683ddd4a05a...|2019-09-12T00:00:...|http://example.co...|user-043| NE| Safari|windows|403|0.2843|
|a45df30287454e598...|2019-09-12T00:01:...|http://example.co...|user-005| VI|   Edge|Android|444|0.6983|
|941c2d880242431c9...|2019-09-12T00:01:...|http://example.co...|user-092| IS|Firefox|windows|226|0.2636|
|97723d7e1669450eb...|2019-09-12T00:02:...|http://example.co...|user-024| GM|Firefox|windows|303|0.0459|
+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
only showing top 5 rows



In [18]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("allData")

In [71]:
avgTtfbDF = spark.sql("""
WITH cte AS
    ( SELECT SUBSTRING(_c1,1,10) AS day, _c2 AS url, _c8 AS ttfb 
    FROM allData )
SELECT day,url,AVG(ttfb) as avg_ttfb FROM cte
GROUP BY day,url
ORDER BY day,avg_ttfb,url
""")
avgTtfbDF.show(10)

+----------+--------------------+-------------------+
|       day|                 url|           avg_ttfb|
+----------+--------------------+-------------------+
|2019-09-12|http://example.co...| 0.3931014084507042|
|2019-09-12|http://example.co...|           0.402545|
|2019-09-12|http://example.co...|0.41331718750000007|
|2019-09-12|http://example.co...| 0.4188678571428572|
|2019-09-12|http://example.co...|0.41928939393939396|
|2019-09-12|http://example.co...|0.42255818181818183|
|2019-09-12|http://example.co...|  0.428484126984127|
|2019-09-12|http://example.co...| 0.4292042857142857|
|2019-09-12|http://example.co...| 0.4307040816326531|
|2019-09-12|http://example.co...| 0.4317596774193549|
+----------+--------------------+-------------------+
only showing top 10 rows



In [72]:
# Register the DataFrame as a SQL temporary view
avgTtfbDF.createOrReplaceTempView("avgData")

In [73]:
spark.sql("""
WITH cte AS
  ( SELECT day, url, avg_ttfb,
           ROW_NUMBER() OVER (PARTITION BY day ORDER BY avg_ttfb ASC) AS ttfb_rank
    FROM avgData )
SELECT day, url, avg_ttfb FROM cte
WHERE ttfb_rank <= 5
ORDER BY day,avg_ttfb
""").show()

+----------+--------------------+-------------------+
|       day|                 url|           avg_ttfb|
+----------+--------------------+-------------------+
|2019-09-12|http://example.co...| 0.3931014084507042|
|2019-09-12|http://example.co...|           0.402545|
|2019-09-12|http://example.co...|0.41331718750000007|
|2019-09-12|http://example.co...| 0.4188678571428572|
|2019-09-12|http://example.co...|0.41928939393939396|
|2019-09-13|http://example.co...|0.40281428571428574|
|2019-09-13|http://example.co...| 0.4294693548387097|
|2019-09-13|http://example.co...| 0.4306358490566038|
|2019-09-13|http://example.co...| 0.4313811594202898|
|2019-09-13|http://example.co...| 0.4319064516129032|
|2019-09-14|http://example.co...|0.40559807692307687|
|2019-09-14|http://example.co...| 0.4185893333333333|
|2019-09-14|http://example.co...|0.42331587301587303|
|2019-09-14|http://example.co...| 0.4265705882352941|
|2019-09-14|http://example.co...|0.42943692307692305|
|2019-09-15|http://example.c

In [56]:
# Register the DataFrame as a SQL temporary view
ttfbDF.createOrReplaceTempView("ttfbData")

In [70]:
avgTtfbDF = spark.sql("""
SELECT day,url,AVG(ttfb) as avg_ttfb FROM ttfbData
GROUP BY day,url
ORDER BY day,url,avg_ttfb
""")
avgTtfbDF.show(10)

+----------+--------------------+-------------------+
|       day|                 url|           avg_ttfb|
+----------+--------------------+-------------------+
|2019-09-12|http://example.co...|0.47357419354838715|
|2019-09-12|http://example.co...| 0.4880267605633803|
|2019-09-12|http://example.co...| 0.4464488888888889|
|2019-09-12|http://example.co...| 0.4611421874999999|
|2019-09-12|http://example.co...| 0.5223234567901235|
|2019-09-12|http://example.co...| 0.5320376623376624|
|2019-09-12|http://example.co...| 0.5226535714285714|
|2019-09-12|http://example.co...|0.45305135135135133|
|2019-09-12|http://example.co...| 0.4934733333333334|
|2019-09-12|http://example.co...| 0.5582210526315791|
+----------+--------------------+-------------------+
only showing top 10 rows



In [58]:
# Register the DataFrame as a SQL temporary view
avgTtfbDF.createOrReplaceTempView("avgData")

In [59]:
spark.sql("""
WITH cte AS
  ( SELECT day, url, avg_ttfb,
           ROW_NUMBER() OVER (PARTITION BY day ORDER BY avg_ttfb DESC) AS ttfb_rank
    FROM avgData
  )
SELECT day, url, avg_ttfb FROM cte
WHERE ttfb_rank <= 5
ORDER BY day,avg_ttfb
""").show()

+----------+--------------------+------------------+
|       day|                 url|          avg_ttfb|
+----------+--------------------+------------------+
|2019-09-12|http://example.co...|0.5582210526315791|
|2019-09-12|http://example.co...|0.5655351851851852|
|2019-09-12|http://example.co...|0.5661685185185186|
|2019-09-12|http://example.co...|          0.578595|
|2019-09-12|http://example.co...| 0.600576923076923|
|2019-09-13|http://example.co...|0.5711131147540983|
|2019-09-13|http://example.co...|            0.5753|
|2019-09-13|http://example.co...|0.5777212121212122|
|2019-09-13|http://example.co...|0.5834452054794521|
|2019-09-13|http://example.co...|0.5982431034482759|
|2019-09-14|http://example.co...|0.5734783783783786|
|2019-09-14|http://example.co...|0.5754071428571429|
|2019-09-14|http://example.co...|0.5754943661971831|
|2019-09-14|http://example.co...|0.5772901408450704|
|2019-09-14|http://example.co...|0.5933677966101695|
|2019-09-15|http://example.co...|            0

In [60]:
spark.sql("""
SELECT day, url, avg_ttfb, ROW_NUMBER() OVER (PARTITION BY day ORDER BY avg_ttfb DESC) AS ttfb_rank
FROM rankedData
""").show()

+----------+--------------------+------------------+---------+
|       day|                 url|          avg_ttfb|ttfb_rank|
+----------+--------------------+------------------+---------+
|2019-09-13|http://example.co...|0.5982431034482759|        1|
|2019-09-13|http://example.co...|0.5834452054794521|        2|
|2019-09-13|http://example.co...|0.5777212121212122|        3|
|2019-09-13|http://example.co...|            0.5753|        4|
|2019-09-13|http://example.co...|0.5711131147540983|        5|
|2019-09-13|http://example.co...|0.5615442307692308|        6|
|2019-09-13|http://example.co...|0.5552396825396826|        7|
|2019-09-13|http://example.co...|0.5543915662650603|        8|
|2019-09-13|http://example.co...|0.5535523076923078|        9|
|2019-09-13|http://example.co...|0.5522982456140351|       10|
|2019-09-13|http://example.co...|0.5520215189873419|       11|
|2019-09-13|http://example.co...|0.5484666666666667|       12|
|2019-09-13|http://example.co...|0.5481449275362319|   